# Snowflake Observability Demo
## Monitoring UDFs and Stored Procedures with Logs, Traces, and Metrics

**For Understood for All - February 2026 Hands-On Session**

---

### What is Observability?

When you run custom code in Snowflake (User-Defined Functions and Stored Procedures), you need visibility into:
- **What happened?** - Did it succeed or fail?
- **How long did it take?** - Is it running slowly?
- **What resources did it use?** - How much memory/CPU?

Snowflake's **Event Table** automatically captures this information so you can monitor and debug your code.

---

### The Three Types of Telemetry

| Type | What It Captures | How You Emit It | Why It Matters |
|------|-----------------|-----------------|----------------|
| **Logs** | Messages, warnings, errors | Python's `logging` module | Debug issues, track what happened |
| **Traces** | Events and timing within code execution | `snowflake.telemetry` package | Understand execution flow, find bottlenecks |
| **Metrics** | CPU and memory usage | Auto-captured by Snowflake | Monitor resource consumption |

---

### Demo Workflow

In this notebook, we will:

1. **Create a UDF** - A reusable function that categorizes a child's learning support needs
2. **Test the UDF** - Run it with different assessment scenarios
3. **Create a Stored Procedure** - A batch processor that categorizes multiple families
4. **Run the Procedure** - Process sample family intake data
5. **Query the Event Table** - View the logs, traces, and metrics we generated

---

### Why This Matters for Understood

When processing family intake assessments:
- **Without observability:** If something goes wrong, you have no idea why
- **With observability:** You see exactly what happened, when, and why - ensuring families get the support they need

---
## Setup: Connect to Snowflake

First, we connect to our demo environment. This cell:
1. Gets the active Snowflake session (already connected in Snowsight)
2. Switches to our demo database and schema
3. Selects our warehouse for compute

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
session.sql('USE DATABASE UNDERSTOOD_DEMO').collect()
session.sql('USE SCHEMA OBSERVABILITY').collect()
session.sql('USE WAREHOUSE UNDERSTOOD_DEMO_WH').collect()
print('Connected to UNDERSTOOD_DEMO.OBSERVABILITY')

---
## Enable Telemetry Collection

By default, Snowflake doesn't capture all telemetry. We need to enable it:

- **METRIC_LEVEL = ALL** - Capture CPU and memory usage
- **TRACE_LEVEL = ALWAYS** - Capture trace events and spans

This tells Snowflake: "Record everything that happens when my code runs."

In [ ]:
session.sql('ALTER SESSION SET METRIC_LEVEL = ALL').collect()
session.sql('ALTER SESSION SET TRACE_LEVEL = ALWAYS').collect()
print('Telemetry collection enabled!')
print('- Metrics: ALL (CPU and memory will be captured)')
print('- Traces: ALWAYS (all trace events will be recorded)')

---
# Part 1: Create a User-Defined Function (UDF)
## Learning Support Categorizer

---

### What is a UDF?

A **User-Defined Function (UDF)** is custom code that runs inside Snowflake. You can call it from SQL just like built-in functions.

### Our Use Case

When families complete intake assessments for Understood, children receive scores in three areas:
- **Reading Score** - Reading comprehension and fluency
- **Math Score** - Mathematical reasoning
- **Focus Score** - Attention and executive function

This UDF takes those three scores and returns:
1. **Support Level** - How intensive should the support be? (Intensive, Moderate, or Light Touch)
2. **Support Areas** - Which specific areas need attention?

### Observability in This UDF

We add two types of telemetry:

1. **Logs** (`logging.info()`) - Record what's happening in plain text
2. **Traces** (`telemetry.add_event()` and `telemetry.set_span_attribute()`) - Record structured data for analysis

In [ ]:
session.sql('''
CREATE OR REPLACE FUNCTION categorize_support_level(
    reading_score INT,
    math_score INT,
    focus_score INT
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('snowflake-telemetry-python')
HANDLER = 'categorize'
AS $$
import logging
import json
from snowflake import telemetry

logger = logging.getLogger("support_categorizer")

def categorize(reading_score, math_score, focus_score):
    # LOG: Record that we started processing
    logger.info(json.dumps({
        "event": "categorization_started",
        "reading": reading_score,
        "math": math_score,
        "focus": focus_score
    }))
    
    # TRACE: Record input values as span attributes
    telemetry.set_span_attribute("input.reading_score", reading_score)
    telemetry.set_span_attribute("input.math_score", math_score)
    telemetry.set_span_attribute("input.focus_score", focus_score)
    
    # Business logic: Determine which areas need support
    areas = []
    if reading_score < 70:
        areas.append("Reading")
    if math_score < 70:
        areas.append("Math")
    if focus_score < 70:
        areas.append("Focus")
    
    # Business logic: Calculate overall support level
    avg = (reading_score + math_score + focus_score) / 3
    if avg < 50:
        level = "Intensive"
    elif avg < 70:
        level = "Moderate"
    else:
        level = "Light Touch"
    
    result = f"{level}: {', '.join(areas) if areas else 'General Support'}"
    
    # TRACE: Add an event marking completion with details
    telemetry.add_event("categorization_complete", {
        "level": level,
        "areas_count": len(areas),
        "avg_score": avg
    })
    
    # LOG: Record the final result
    logger.info(json.dumps({
        "event": "categorization_complete",
        "result": result
    }))
    
    # TRACE: Record the output as a span attribute
    telemetry.set_span_attribute("output.result", result)
    
    return result
$$
''').collect()

print('UDF created successfully!')
print('')
print('This function now emits:')
print('  - LOGS: Start and completion messages with details')
print('  - TRACES: Input scores, output result, and completion event')
print('  - METRICS: CPU and memory usage (automatic)')

---
## Test the UDF

Let's test our function with three different scenarios that Understood might encounter:

| Scenario | Reading | Math | Focus | Expected Result |
|----------|---------|------|-------|----------------|
| High-performing child | 85 | 90 | 80 | Light Touch - no specific areas |
| Mixed needs | 55 | 75 | 60 | Moderate - Reading and Focus support |
| Intensive support needed | 40 | 45 | 35 | Intensive - all areas need help |

In [ ]:
print('Testing the UDF with different assessment scenarios:')
print('=' * 60)

tests = [
    (85, 90, 80, 'High-performing child'),
    (55, 75, 60, 'Mixed needs'),
    (40, 45, 35, 'Intensive support needed'),
]

for reading, math, focus, description in tests:
    result = session.sql(f'SELECT categorize_support_level({reading}, {math}, {focus})').collect()
    print(f'\n{description}:')
    print(f'  Scores: Reading={reading}, Math={math}, Focus={focus}')
    print(f'  Result: {result[0][0]}')

---
# Part 2: Create a Stored Procedure
## Family Intake Processor

---

### What is a Stored Procedure?

A **Stored Procedure** is a program that runs inside Snowflake and can:
- Read and write data
- Call other functions (like our UDF)
- Perform complex multi-step operations

### Our Use Case

When new families sign up with Understood, their information goes into an intake table. This procedure:

1. **Finds** all families that haven't been categorized yet
2. **Calls** our UDF to determine their support needs
3. **Updates** each family's record with their support category
4. **Logs** every step so we can track what happened

### Why This Matters

In a real system, this might run automatically every night to process new signups. If something goes wrong, the observability data helps you:
- See which families were processed
- Identify any errors
- Understand performance (how long did it take?)

---
## Step 2a: Create Sample Family Data

First, we create a table to store family intake information. In a real system, this data would come from:
- Online signup forms
- Partner organization referrals
- School district integrations

For this demo, we'll insert 5 sample families with different assessment scores.

In [ ]:
session.sql('''
CREATE OR REPLACE TABLE family_intakes (
    family_id INT,
    family_name STRING,
    child_age INT,
    reading_score INT,
    math_score INT,
    focus_score INT,
    support_category STRING,
    processed_at TIMESTAMP
)
''').collect()

session.sql('''
INSERT INTO family_intakes (family_id, family_name, child_age, reading_score, math_score, focus_score)
VALUES 
    (1, 'Johnson Family', 8, 65, 80, 55),
    (2, 'Chen Family', 10, 45, 50, 60),
    (3, 'Garcia Family', 7, 85, 90, 88),
    (4, 'Wilson Family', 9, 40, 35, 45),
    (5, 'Brown Family', 11, 70, 65, 72)
''').collect()

print('Created family_intakes table with 5 sample families:')
print('')
session.sql('SELECT * FROM family_intakes').to_pandas()

---
## Step 2b: Create the Stored Procedure

This procedure processes all unprocessed families. Notice how we add observability at each step:

| Step | What Happens | Observability Added |
|------|-------------|---------------------|
| Start | Begin processing | LOG + TRACE: "batch_started" |
| Find Records | Query for NULL support_category | LOG + TRACE: record count |
| Process Each | Call UDF, update record | LOG + TRACE: family details |
| Complete | Finish batch | LOG + TRACE: total processed |

**Note:** Metrics (CPU/memory) are captured automatically - we don't need to write any code for that!

In [ ]:
session.sql('''
CREATE OR REPLACE PROCEDURE process_family_intakes()
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.10'
PACKAGES = ('snowflake-snowpark-python', 'snowflake-telemetry-python')
HANDLER = 'process'
AS $$
import logging
import json
from snowflake.snowpark import Session
from snowflake import telemetry

logger = logging.getLogger("intake_processor")

def process(session: Session):
    # STEP 1: Mark the start of processing
    logger.info(json.dumps({"event": "batch_started"}))
    telemetry.set_span_attribute("batch.status", "started")
    telemetry.add_event("batch_processing_started")
    
    # STEP 2: Find all unprocessed families
    unprocessed = session.sql("""
        SELECT family_id, family_name, reading_score, math_score, focus_score 
        FROM family_intakes 
        WHERE support_category IS NULL
    """).collect()
    
    record_count = len(unprocessed)
    logger.info(json.dumps({"event": "records_found", "count": record_count}))
    telemetry.set_span_attribute("batch.record_count", record_count)
    
    # STEP 3: Process each family
    processed = 0
    for row in unprocessed:
        fid = row["FAMILY_ID"]
        fname = row["FAMILY_NAME"]
        
        cat_result = session.sql(f"""
            SELECT categorize_support_level(
                {row["READING_SCORE"]}, 
                {row["MATH_SCORE"]}, 
                {row["FOCUS_SCORE"]}
            )
        """).collect()[0][0]
        
        session.sql(f"""
            UPDATE family_intakes 
            SET support_category = '{cat_result}',
                processed_at = CURRENT_TIMESTAMP()
            WHERE family_id = {fid}
        """).collect()
        
        logger.info(json.dumps({
            "event": "family_processed",
            "family_id": fid,
            "family_name": fname,
            "category": cat_result
        }))
        
        telemetry.add_event("family_processed", {
            "family_id": fid,
            "category": cat_result
        })
        
        processed += 1
    
    # STEP 4: Mark completion
    logger.info(json.dumps({"event": "batch_complete", "total": processed}))
    telemetry.set_span_attribute("batch.processed_count", processed)
    telemetry.set_span_attribute("batch.status", "completed")
    telemetry.add_event("batch_processing_complete", {"total_processed": processed})
    
    return f"Processed {processed} families"
$$
''').collect()

print('Stored Procedure created successfully!')
print('')
print('This procedure emits telemetry at every step:')
print('  1. Batch start (log + trace)')
print('  2. Records found (log + trace with count)')
print('  3. Each family processed (log + trace with details)')
print('  4. Batch complete (log + trace with total)')
print('  5. CPU/Memory metrics (automatic)')

---
## Step 2c: Run the Stored Procedure

Now let's execute our procedure. This will:
1. Find the 5 unprocessed families
2. Categorize each one using our UDF
3. Update the database with results
4. Generate telemetry data we can analyze

In [ ]:
print('Running the family intake processor...')
print('')

result = session.sql('CALL process_family_intakes()').collect()

print(f'Result: {result[0][0]}')
print('')
print('The procedure has now generated logs, traces, and metrics!')

---
## Step 2d: View the Processed Families

Let's see the results. Each family should now have:
- A **support_category** (the result from our UDF)
- A **processed_at** timestamp

In [ ]:
print('Processed family records:')
print('')
session.sql('SELECT * FROM family_intakes ORDER BY family_id').to_pandas()

---
# Part 3: Query the Event Table
## Viewing Our Telemetry Data with SQL

---

All the telemetry we generated (logs, traces, metrics) is stored in an **Event Table**.

Our event table is: `UNDERSTOOD_DEMO.OBSERVABILITY.DEMO_EVENTS`

### Key Columns in the Event Table

| Column | What It Contains |
|--------|------------------|
| `TIMESTAMP` | When the event occurred |
| `RECORD_TYPE` | Type: 'LOG', 'SPAN', 'SPAN_EVENT', or 'METRIC' |
| `SCOPE['name']` | Logger name (e.g., 'support_categorizer') |
| `VALUE` | The log message or event data |
| `RECORD_ATTRIBUTES` | Custom attributes we added |

In [ ]:
import time
print('Waiting for telemetry data to be available...')
time.sleep(5)
print('Ready to query!')

---
## View LOGS

Logs are messages we wrote with `logger.info()`. They show:
- When each event happened
- Which logger wrote them
- The message content

**Filter:** `RECORD_TYPE = 'LOG'`

In [ ]:
logs = session.sql('''
SELECT 
    TIMESTAMP,
    SCOPE['name'] as LOGGER_NAME,
    RECORD['severity_text'] as LOG_LEVEL,
    VALUE as MESSAGE
FROM UNDERSTOOD_DEMO.OBSERVABILITY.DEMO_EVENTS
WHERE RECORD_TYPE = 'LOG'
  AND SCOPE['name'] IN ('support_categorizer', 'intake_processor')
  AND TIMESTAMP > DATEADD(minute, -15, CURRENT_TIMESTAMP())
ORDER BY TIMESTAMP DESC
LIMIT 25
''').to_pandas()

print(f'Found {len(logs)} log entries:')
print('')
logs

---
## View TRACE EVENTS

Trace events are moments we recorded with `telemetry.add_event()`.

**Filter:** `RECORD_TYPE = 'SPAN_EVENT'`

In [ ]:
traces = session.sql('''
SELECT 
    TIMESTAMP,
    RECORD['name'] as EVENT_NAME,
    RECORD_ATTRIBUTES as EVENT_DETAILS
FROM UNDERSTOOD_DEMO.OBSERVABILITY.DEMO_EVENTS
WHERE RECORD_TYPE = 'SPAN_EVENT'
  AND TIMESTAMP > DATEADD(minute, -15, CURRENT_TIMESTAMP())
ORDER BY TIMESTAMP DESC
LIMIT 20
''').to_pandas()

print(f'Found {len(traces)} trace events:')
print('')
traces

---
## View SPAN ATTRIBUTES

Spans represent function execution. The attributes we added with `telemetry.set_span_attribute()` appear here.

**Filter:** `RECORD_TYPE = 'SPAN'`

In [ ]:
spans = session.sql('''
SELECT 
    TIMESTAMP,
    RESOURCE_ATTRIBUTES['snow.executable.name'] as FUNCTION_NAME,
    RECORD_ATTRIBUTES as CUSTOM_ATTRIBUTES
FROM UNDERSTOOD_DEMO.OBSERVABILITY.DEMO_EVENTS
WHERE RECORD_TYPE = 'SPAN'
  AND TIMESTAMP > DATEADD(minute, -15, CURRENT_TIMESTAMP())
  AND RECORD_ATTRIBUTES IS NOT NULL
ORDER BY TIMESTAMP DESC
LIMIT 15
''').to_pandas()

print(f'Found {len(spans)} spans with custom attributes:')
print('')
spans

---
## View METRICS

Metrics capture resource usage (CPU and memory) automatically. No code required!

**Filter:** `RECORD_TYPE = 'METRIC'`

In [ ]:
metrics = session.sql('''
SELECT 
    TIMESTAMP,
    RESOURCE_ATTRIBUTES['snow.executable.name'] as FUNCTION_NAME,
    RECORD['metric']['name'] as METRIC_NAME,
    RECORD['metric']['value'] as METRIC_VALUE
FROM UNDERSTOOD_DEMO.OBSERVABILITY.DEMO_EVENTS
WHERE RECORD_TYPE = 'METRIC'
  AND TIMESTAMP > DATEADD(minute, -15, CURRENT_TIMESTAMP())
ORDER BY TIMESTAMP DESC
LIMIT 20
''').to_pandas()

print(f'Found {len(metrics)} metric entries:')
print('')
metrics

---
# Summary: What We Learned

## The Complete Workflow

| Step | What We Did | Observability Output |
|------|------------|---------------------|
| 1. Created UDF | `categorize_support_level()` function | Logs + Traces for each categorization |
| 2. Tested UDF | Ran with 3 different scenarios | Generated telemetry for each test |
| 3. Created Data | Sample family intake records | - |
| 4. Created Procedure | `process_family_intakes()` batch processor | Logs + Traces at every step |
| 5. Ran Procedure | Processed all 5 families | Telemetry for batch + each family |
| 6. Queried Event Table | Viewed logs, traces, spans, metrics | Saw all our observability data |

---

## Key Takeaways for Understood

### Why This Matters
- **Debug issues faster:** When a family isn't processed correctly, trace back through logs
- **Monitor performance:** CPU/memory metrics show if processing is efficient
- **Audit trail:** Every action is recorded with timestamps and details

### How to Use This in Production
1. **UDFs** - For reusable business logic (categorization, scoring, validation)
2. **Stored Procedures** - For batch operations (nightly processing, data pipelines)
3. **Event Table** - Query with SQL for debugging, monitoring, and reporting

### Observability Methods

| Method | What It Captures | When to Use |
|--------|-----------------|-------------|
| `logging.info()` | Text messages | Debugging, status updates |
| `telemetry.set_span_attribute()` | Key-value data | Input/output values, IDs |
| `telemetry.add_event()` | Named events with data | Milestones, completion markers |
| *(automatic)* | CPU/memory metrics | Always captured when enabled |